<a href="https://colab.research.google.com/github/TheGreatestEvers/ADL4CV/blob/dev_evaluation/zero-shot/test_evaluation_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Clone Private GitHub Repository
import os
from getpass import getpass

# Replace with the repository URL
repo_url = "https://github.com/TheGreatestEvers/ADL4CV.git"

# Enter your GitHub username
username = "maxdoesch"

# Enter your personal access token
token = getpass('Enter your GitHub personal access token: ')

# Construct the full repository URL with the token
repo_url_with_token = repo_url.replace("https://", f"https://{username}:{token}@")

# Clone the repository
!git clone -b dev_evaluation {repo_url_with_token}


Enter your GitHub personal access token: ··········
Cloning into 'ADL4CV'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 109 (delta 64), reused 81 (delta 41), pack-reused 0
Receiving objects: 100% (109/109), 3.77 MiB | 17.55 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [2]:
%cd ADL4CV
!pip install -r requirements.txt

/content/ADL4CV
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 32.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-p

In [3]:
!wget https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
!unzip tapvid_davis.zip
%cd zero-shot

--2024-05-19 15:43:08--  https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.207, 108.177.121.207, 108.177.120.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.119.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1668710491 (1.6G) [application/zip]
Saving to: ‘tapvid_davis.zip’

tapvid_davis.zip    100%[===================>]   1.55G   110MB/s    in 9.4s    

2024-05-19 15:43:18 (169 MB/s) - ‘tapvid_davis.zip’ saved [1668710491/1668710491]

Archive:  tapvid_davis.zip
   creating: tapvid_davis/
  inflating: tapvid_davis/SOURCES.md  
  inflating: tapvid_davis/README.md  
  inflating: tapvid_davis/tapvid_davis.pkl  
/content/ADL4CV/zero-shot


In [4]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper()
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
from evaluation_datasets import create_davis_dataset

davis_dataset = create_davis_dataset('../tapvid_davis/tapvid_davis.pkl')

first_video = {}

for data in davis_dataset:
    first_video = data['davis']
    break

print(first_video['video'].dtype)

video_tensor = torch.tensor(first_video['video'])
print(video_tensor.shape)

uint8
torch.Size([1, 90, 256, 256, 3])


In [6]:
video_features_dict = diffusion_wrapper.extract_video_features(video_tensor, "")

In [7]:
video_features = diffusion_wrapper.concatenate_video_features({'up_block': video_features_dict['up_block'][1:3]})
print(video_features.shape)

torch.Size([90, 1920, 32, 32])


In [8]:
video_features = video_features.permute(0, 2, 3, 1).float()
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
tracks = zero_shot_tracker.track(heatmaps)

In [9]:
zero_shot_tracker.place_marker_in_frames(first_video['video'].squeeze(), tracks)
heatmap_generator.safe_heatmap_as_gif(heatmaps)